## Supervised machine learning

Having explored some unsupervised machine learning methods earlier, we will move on to supervised techniques. Supervised learning is the task of finding a mapping from input data to output labels based on example input-output pairs. During the training (or fitting) phase, the model builds an internal function which maps input to output, ideally in a way that can be generalized to yet-unseen data successfully. Once a model is trained, it can be used to predict output labels from new input data alone.

They come in two main groups: classifiers for categorical labels, and regressors for continuous numeric labels.

Our first dataset, like yesterday, will be the mice gene expression data on the input side, and the categorical diet labels on the output (also referred to as "target") side. The first supervised model that we will train and evaluate will be a support vector classifier.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA

We will load the expression and diet label data the same way as we did yesterday, and again, we will transpose the table so that samples correspond to rows, and the so-called "features" correspond to columns. From now on, instead of `True/False` we will use `1/0` when encoding the diet target labels, for compatibility with a larger range of models.

In [ ]:
xls = pd.read_excel('../data-livermito/aad0189_DataFileS5.xlsx', header=2)
expr = xls.groupby('Gene.1').mean()
expr = expr.loc[:, expr.columns.str.contains('Liver')]
is_hfd = pd.Series(expr.columns.str.contains('HFD'), index=expr.columns)  # diet labels for verification

data = expr.T    # use the name 'data' for input
target = 1 * is_hfd  # use the name 'target' for the labels / expected outputs. Let's convert it to 1's and 0's.

## Task 1: Create a support vector classifier (SVC) using a linear kernel

### 1.1: Train it with the whole dataset
Use a simple linear kernel for the SVC.

### 1.2: Evaluate its accuracy on the training data
We are asking you to commit a **mortal sin**: evaluate a machine learning algorithm on the same data that you had trained it with. But do it anyway. Are you surprised by its accuracy?

### 1.3: Compare the support vector with the 4 direction vectors from yesterday's PCA
To save you the trouble of moving code between sheets, we provided you the PCA code below, and put the principal direction vectors in a DataFrame. Add the linear support vector, find a suitable way of comparing it with the principal direction vectors, and discuss what you found.

In [ ]:
pca = PCA(n_components=4)
pca.fit(data)
coeffs = pd.DataFrame(pca.components_.T, columns=['pc1', 'pc2', 'pc3', 'pc4'])
# coeffs['linear_svc'] = ...

### 1.4: Validate the model in a legitimate way
In 1.2 we asked you to evaluate the model with the same data you had trained it with. It's a poor and frowned upon way to test a model's accuracy, since you can't tell whether the model has merely memorized the exact inputs and expected outputs, or whether it truly found generalizable patterns in the data. The former scenario is often called *overfitting*.

This time, train the model with one half of the data, and validate its accuracy on the other half. It will ensure that the test data will be new for the classifier, and it can't simply recall their output labels.

Keep it simple for now, rows `::2` for training and `1::2` for validating should be good enough.

### 1.4.2: Reduce the number of features
You may have found that the model still performs excellent, and it was successful at learning the structure of the data instead of just memorizing the training data. This a good sign, and it shows that it's not challenging for an SVM to distinguish the two diets from the full expression dataset. Let's make the SVM sweat a bit.

Try cutting down the number of used features until the point that validation is no longer fully accurate. Keep only every N-th feature (e.g. train and validate with `data.loc[::2,::N]` and `data.loc[1::2, ::N]`) until the model starts making mistakes. You can simply re-run the same jupyter cell with increasing N-s: we don't expect a graph or exhaustive measurements here, just find one large enough N that the accuracy isn't perfect anymore.

### 1.5: Perform cross-validation
Previously you had separated your data into a disjoint training and testing set, trained the model with the training set and computed its accuracy on the testing set. But you could also switch them around (train with `1::2`, test with `::2`) and compute a second accuracy value. The average of those two values would give you an even better estimate of the real accuracy. This would be called a *2-fold cross-validation*.

3-fold cross-validation is when you split your data to D1/D2/D3 thirds, train the model with D1+D2 and test on D3. Then train with D1+D3 and test on D2, finally train with D2+D3 and test on D1. The average of the three accuracy values would be your model's 3-fold cross-validated accuracy.

(Sidenote: the *model* is the SVM with whatever parameters it has (linear + defaults) and the features that you use. During cross-validation you test different *instances* of the same model: you train them with different data so they will all end up slightly different, you test them with different data so the accuracy estimates will also be different, but you are validating the same *model* all along.)

`sklearn` gives you helper classes to split your data into training and testing sets following a cross-validation scheme, saving you most of the legwork. Some classes even support features like random shuffling, or stratification (keeping labels balanced across training/testing classes). Browse the documentation, choose one that you like, and cross-validate your model on the restricted amount of features chosen in 1.4.2.

### 1.6: Optional: Play around

Ideas: Test the limits of the linear SVM by choosing an increasing number of random features, train and cross-validate multiple SVM instances for each, and plot the average accuracy as a function the number of features used. Try different kernels and parameters. Test robustness: add noise to the data, or introduce a few random mistakes in the training labels.